# **Split Dataset**

This was used the divide the subset of images from FFHQ and the ones that had the mask added into training, testing, and validation sets. *It will not run now that the images have been sorted. 

## Mount Google Drive

Mounting the Drive allows the notebook to access the dataset folder. You can see all the files in your Drive this way, but no one else can see them because a new VM is loaded each time. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Set paths and import packages

In [ ]:
import shutil
import glob
import numpy as np
import pandas as pd

In [ ]:
unmasked_path = 'gdrive/My Drive/CV-FinalProject/dataset/FFHQ_subset/'
masked_path = 'gdrive/My Drive/CV-FinalProject/dataset/FFHQ_subset_masked/'
validation_path = 'gdrive/My Drive/CV-FinalProject/dataset/validation/'
test_path = 'gdrive/My Drive/CV-FinalProject/dataset/test/'
training_path = 'gdrive/My Drive/CV-FinalProject/dataset/training/'

In [ ]:
def group(arr, size):
  arr1 = np.random.choice(arr, size, replace=False)
  for i in list(arr1):
    if i in arr:
      arr.remove(i)
  return list(arr1), arr

In [ ]:
def move_masked(array, outdir):
  names = []
  for a in array:
    names.append(a.split('/')[-1])
    shutil.move(a, outdir)
  return names

In [ ]:
def move_unmasked(array, indir, outdir):
  names = []
  for a in array:
    name = a.split('/')[-1].split('_')[0] + '.png'
    names.append(name)
    shutil.move((indir + name), outdir)
  return names

In [ ]:
#load the masked file names
nums = [fname for fname in glob.glob((masked_path + '/*.png'))]
test_size = round(len(nums)*.1)

In [ ]:
#divide the numbers into masked and unmasked
test, nums = group(nums, test_size)
validation, train = group(nums, test_size)

In [ ]:
#split each set into masked and unmasked
test_masked, test_unmasked = group(test, round(len(test)/2))
validation_masked, validation_unmasked = group(validation, round(len(validation)/2))
train_masked, train_unmasked = group(train, round(len(train)/2))

In [ ]:
#move the testing data
files = move_unmasked(test_unmasked, unmasked_path, test_path)
df = pd.DataFrame({'files': files, 'class': np.zeros(len(files), dtype='int')})
files = move_masked(test_masked, test_path)
df = df.append(pd.DataFrame({'files':files, 'class': np.ones(len(files), dtype='int')}))
df.to_csv((test_path + 'test.csv'))

In [ ]:
#move the validation data
files = move_unmasked(validation_unmasked, unmasked_path, validation_path)
df = pd.DataFrame({'files': files, 'class': np.zeros(len(files), dtype='int')})
files = move_masked(validation_masked, validation_path)
df = df.append(pd.DataFrame({'files':files, 'class': np.ones(len(files), dtype='int')}))
df.to_csv((validation_path + 'validation.csv'))

In [ ]:
#move the training data
files = move_unmasked(train_unmasked, unmasked_path, training_path)
df = pd.DataFrame({'files': files, 'class': np.zeros(len(files), dtype='int')})
files = move_masked(train_masked, training_path)
df = df.append(pd.DataFrame({'files': files, 'class': np.ones(len(files), dtype='int')}))
df.to_csv((training_path + 'training.csv'))